In [35]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.applications.resnet50 import decode_predictions
from tensorflow.keras.applications.resnet50 import preprocess_input
import tensorflow as tf
import numpy as np
import json
import cv2
import os

In [14]:
print(tf.test.is_gpu_available())
print(tf.config.list_physical_devices('GPU'))

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
True
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [15]:
# instantiate globals
EPS = .001
LR = 0.1
sccLoss = SparseCategoricalCrossentropy()
optimizer = Adam(learning_rate=LR)

In [16]:
with open("imagenet_class_index.json", "r") as jf:
    class_index_json = json.load(jf)
print(class_index_json)

{'0': ['n01440764', 'tench'], '1': ['n01443537', 'goldfish'], '2': ['n01484850', 'great_white_shark'], '3': ['n01491361', 'tiger_shark'], '4': ['n01494475', 'hammerhead'], '5': ['n01496331', 'electric_ray'], '6': ['n01498041', 'stingray'], '7': ['n01514668', 'cock'], '8': ['n01514859', 'hen'], '9': ['n01518878', 'ostrich'], '10': ['n01530575', 'brambling'], '11': ['n01531178', 'goldfinch'], '12': ['n01532829', 'house_finch'], '13': ['n01534433', 'junco'], '14': ['n01537544', 'indigo_bunting'], '15': ['n01558993', 'robin'], '16': ['n01560419', 'bulbul'], '17': ['n01580077', 'jay'], '18': ['n01582220', 'magpie'], '19': ['n01592084', 'chickadee'], '20': ['n01601694', 'water_ouzel'], '21': ['n01608432', 'kite'], '22': ['n01614925', 'bald_eagle'], '23': ['n01616318', 'vulture'], '24': ['n01622779', 'great_grey_owl'], '25': ['n01629819', 'European_fire_salamander'], '26': ['n01630670', 'common_newt'], '27': ['n01631663', 'eft'], '28': ['n01632458', 'spotted_salamander'], '29': ['n01632777', 

In [17]:
class_indices = {}
for k, v in class_index_json.items():
    class_indices[v[0]] = v[1]
print(class_indices)

{'n01440764': 'tench', 'n01443537': 'goldfish', 'n01484850': 'great_white_shark', 'n01491361': 'tiger_shark', 'n01494475': 'hammerhead', 'n01496331': 'electric_ray', 'n01498041': 'stingray', 'n01514668': 'cock', 'n01514859': 'hen', 'n01518878': 'ostrich', 'n01530575': 'brambling', 'n01531178': 'goldfinch', 'n01532829': 'house_finch', 'n01534433': 'junco', 'n01537544': 'indigo_bunting', 'n01558993': 'robin', 'n01560419': 'bulbul', 'n01580077': 'jay', 'n01582220': 'magpie', 'n01592084': 'chickadee', 'n01601694': 'water_ouzel', 'n01608432': 'kite', 'n01614925': 'bald_eagle', 'n01616318': 'vulture', 'n01622779': 'great_grey_owl', 'n01629819': 'European_fire_salamander', 'n01630670': 'common_newt', 'n01631663': 'eft', 'n01632458': 'spotted_salamander', 'n01632777': 'axolotl', 'n01641577': 'bullfrog', 'n01644373': 'tree_frog', 'n01644900': 'tailed_frog', 'n01664065': 'loggerhead', 'n01665541': 'leatherback_turtle', 'n01667114': 'mud_turtle', 'n01667778': 'terrapin', 'n01669191': 'box_turtle'

In [18]:
def preprocess_image(image):
	# image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
	image = cv2.resize(image, (224, 224))
	image = np.expand_dims(image, axis=0)
	return image

In [19]:
def clip_eps(tensor, eps):
	return tf.clip_by_value(tensor, clip_value_min=-eps,
		clip_value_max=eps)

In [20]:
def generate_adversaries(model, baseImage, delta, classIdx, steps=50):
	for step in range(0, steps):
		# track gradients
		with tf.GradientTape() as tape:
			# track perturbation vector for gradient updates
			tape.watch(delta)
			# add our perturbation vector to the base image 
			adversary = preprocess_input(baseImage + delta)
			# run perturbated image through model and calculate the loss with respect to the class index
			predictions = model(adversary, training=False)
			loss = -sccLoss(tf.convert_to_tensor([classIdx]),
				predictions)
			# log the loss
			if step % 5 == 0:
				print("step: {}, loss: {}...".format(step,
					loss.numpy()))
		# calculate the gradients of loss with respect to the
		# perturbation vector
		gradients = tape.gradient(loss, delta)
		# update the weights, clip the perturbation vector, and update its value
		optimizer.apply_gradients([(gradients, delta)])
		delta.assign_add(clip_eps(delta, eps=EPS))
	return delta

In [21]:
image_path = tf.keras.utils.get_file('YellowLabradorLooking_new.jpg', 'https://storage.googleapis.com/download.tensorflow.org/example_images/YellowLabradorLooking_new.jpg')
# image_raw = tf.io.read_file(image_path)
# image = tf.image.decode_image(image_raw)
image = cv2.imread(image_path)
image = preprocess_image(image)

model = ResNet50(weights="imagenet")

baseImage = tf.constant(image, dtype=tf.float32)
cv2.imwrite("temppics\\input.png", baseImage.numpy().squeeze())

delta = tf.Variable(tf.zeros_like(baseImage), trainable=True)
print("[INFO] generating perturbation...")
deltaUpdated = generate_adversaries(model, baseImage, delta, 208)

print("[INFO] creating adversarial example...")
# save perturbation * 50 intensity
cv2.imwrite("temppics\\change.png", (deltaUpdated * 50).numpy().squeeze())
adverImage = (baseImage + deltaUpdated).numpy().squeeze()
# adverImage = np.clip(adverImage, 0, 255).astype("uint8")
# adverImage = cv2.cvtColor(adverImage, cv2.COLOR_RGB2BGR)
cv2.imwrite("temppics\\output.png", adverImage)

# run inference with this adversarial example
print("[INFO] running inference on the adversarial example...")
preprocessedImage = preprocess_input(baseImage + deltaUpdated)
predictions = model.predict(preprocessedImage)
predictions = decode_predictions(predictions, top=3)[0]
print(predictions)
label = predictions[0][1]
confidence = predictions[0][2] * 100
print("[INFO] label: {} confidence: {:.2f}%".format(label,
	confidence))

[INFO] generating perturbation...
step: 0, loss: -2.279902935028076...
step: 5, loss: -10.047632217407227...
step: 10, loss: -19.215961456298828...
step: 15, loss: -28.85980987548828...
step: 20, loss: -37.646976470947266...
step: 25, loss: -46.460323333740234...
step: 30, loss: -55.65617370605469...
step: 35, loss: -65.5890121459961...
step: 40, loss: -75.1854019165039...
step: 45, loss: -84.25175476074219...
[INFO] creating adversarial example...
[INFO] running inference on the adversarial example...
1/1 [==============================] - 1s 588ms/step
[('n02091831', 'Saluki', 1.0), ('n02091134', 'whippet', 5.6281507e-12), ('n02090622', 'borzoi', 1.1388395e-13)]
[INFO] label: Saluki confidence: 100.00%


In [33]:
def predict_normal_image(img_path, model):
    processed_img = cv2.imread(img_path)
    processed_img = preprocess_image(processed_img)
    preds = model.predict(processed_img)
    return decode_predictions(preds, top=1)[0]

In [34]:
print(predict_normal_image(image_path, model))

1/1 [==============================] - 0s 25ms/step
[('n02099712', 'Labrador_retriever', 0.44575375)]


In [71]:
def generate_adv_sample(image_path, filename, classname):
    image = cv2.imread(image_path)
    image = preprocess_image(image)

    model = ResNet50(weights="imagenet")

    baseImage = tf.constant(image, dtype=tf.float32)
    cv2.imwrite(f'results\\inputs\\{classname}\\{filename}', baseImage.numpy().squeeze())

    delta = tf.Variable(tf.zeros_like(baseImage), trainable=True)
    deltaUpdated = generate_adversaries(model, baseImage, delta, 208)

    # save perturbation * 50 intensity
    cv2.imwrite(f'results\\changes\\{classname}\\{filename}', (deltaUpdated * 50).numpy().squeeze())
    adverImage = (baseImage + deltaUpdated).numpy().squeeze()
    # adverImage = np.clip(adverImage, 0, 255).astype("uint8")
    # adverImage = cv2.cvtColor(adverImage, cv2.COLOR_RGB2BGR)
    cv2.imwrite(f'results\\outputs\\{classname}\\{filename}', adverImage)

    # run inference with this adversarial example
    preprocessedImage = preprocess_input(baseImage + deltaUpdated)
    predictions = model.predict(preprocessedImage)
    prediction = decode_predictions(predictions, top=1)[0][0]
    assert prediction != classname
    # predictions = decode_predictions(predictions, top=3)[0]
    # print(predictions)
    # label = predictions[0][1]
    # confidence = predictions[0][2] * 100
    # print("[INFO] label: {} confidence: {:.2f}%".format(label,
    #     confidence))

In [75]:
LIMIT_PER_CLASS = 100
def generate_sample_set(directory_path):
    classfolders = [fol for fol in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, fol))]
    nmodel = ResNet50(weights="imagenet")
    for classname in classfolders:
        cnt = 0
        print(classname)
        sdpath = os.path.join(directory_path, classname)
        files = [os.path.join(sdpath, fil) for fil in os.listdir(sdpath)]
        for file in files:
            filename = file.split("\\")[-1]
            prediction = predict_normal_image(file, nmodel)[0]
            # normal pred is target class
            if prediction[0] == classname:
                generate_adv_sample(file, filename, classname)
                cnt += 1
            if cnt >= LIMIT_PER_CLASS:
                break

In [76]:
# imagenette has classes {tench, English springer, cassette player, chain saw, church, French horn, garbage truck, gas pump, golf ball, parachute}
generate_sample_set("imagenette2\\imagenette2\\train")

n01440764
1/1 [==============================] - 1s 534ms/step
step: 0, loss: -13.033754348754883...
step: 5, loss: -64.38926696777344...
step: 10, loss: -106.38765716552734...
step: 15, loss: -140.6159210205078...
step: 20, loss: -171.5491943359375...
step: 25, loss: -200.00439453125...
step: 30, loss: -227.17340087890625...
step: 35, loss: -250.50257873535156...
step: 40, loss: -269.9913330078125...
step: 45, loss: -287.884521484375...
1/1 [==============================] - 0s 21ms/step
step: 0, loss: -13.345832824707031...
step: 5, loss: -61.33649826049805...
step: 10, loss: -105.21915435791016...
step: 15, loss: -149.10174560546875...
step: 20, loss: -188.75352478027344...
step: 25, loss: -226.8305206298828...
step: 30, loss: -258.415283203125...
step: 35, loss: -286.52264404296875...
step: 40, loss: -312.1611022949219...
step: 45, loss: -333.3772888183594...
1/1 [==============================] - 0s 25ms/step
step: 0, loss: -8.699784278869629...
step: 5, loss: -36.44584655761719..